In [2]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm
import math
import seaborn as sns
import time 
import random
from scipy.interpolate import LSQUnivariateSpline as Spline
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle

In [3]:
# Load data
data = pd.read_csv("../data/full_training_data.csv")
data.head()

,Sold_Price,Bids,Views,Watchers,Auction_Date,Make,Mileage,Exterior Color,Interior Color,Highlights,...,Engine_Cylinders_I6,Engine_Cylinders_V10,Engine_Cylinders_V12,Engine_Cylinders_V6,Engine_Cylinders_V8,Engine_Cylinders_W12,Engine_Cylinders_W8,mod_status_light_mod,mod_status_unknown_mod,Model_Target_Encoded
0,66000.0,26,6277,447.0,"Feb 18, 2026 1:48 PM MST",28,17600.0,13,1,this... is a 2017 ford shelby f-150 supercrew ...,...,False,False,False,False,True,False,False,False,True,43545.115385
1,51000.0,22,11947,962.0,"Feb 18, 2026 1:46 PM MST",5,41000.0,2,0,"this... is a 2009 audi r8 coupe, finished in j...",...,False,False,False,False,True,False,False,False,True,108084.769231
2,17000.0,49,10966,662.0,"Feb 18, 2026 1:38 PM MST",85,310800.0,6,8,"this… is a 2004 toyota land cruiser, finished ...",...,False,False,False,False,True,False,False,False,True,20235.769231
3,29888.0,43,16565,957.0,"Feb 18, 2026 1:36 PM MST",9,NaN,11,1,"this... is a 2007 bmw m5, finished in silverst...",...,False,True,False,False,False,False,False,False,False,29194.666667
4,29995.0,30,8501,585.0,"Feb 18, 2026 1:33 PM MST",73,NaN,6,1,"this... is a 1998 rover mini, finished in brit...",...,False,False,False,False,False,False,False,True,False,29995.000000


In [3]:
# Data Splitting

X = data.drop(columns = ["Sold_Price"])
y = data['Sold_Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=42)

In [4]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test) 

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Training R2: {r2_train:.2f}")
print(f"Test R2:     {r2_test:.2f}")

Training R2: 0.97
Test R2:     0.78


In [7]:
# Saving model and columns
artifacts = {
    "model": rf_model,
    "training_columns": X_train.columns.tolist()
}

with open('../deployment/model_artifacts_002.pkl', 'wb') as f:
    pickle.dump(artifacts, f)
